In [ ]:
from unsloth import FastLanguageModel, Features, Value
import torch
from trl import SFTTrainer
from transformers import TrainingArguments
from datasets import load_dataset

#加载模型
max_seq_length = 2048
dtype = None
load_in_4bit = False
cache_dir = '/root/.cache/modelscope/hub/models'
model_name = f'{cache_dir}/Qwen/Qwen2.5-7B-Instruct'
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
    model_type='qwen2',
    cache_dir=cache_dir
)

==((====))==  Unsloth 2025.3.15: Fast Qwen2 patching. Transformers: 4.49.0.
   \\   /|    NVIDIA A30. Num GPUs = 4. Max memory: 23.486 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Sliding Window Attention is enabled but not implemented for `eager`; unexpected results may be encountered.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
#准备训练数据
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
{}
### Input:
{}
### Response:
{}"""
# 训练数据集
train_dataset_path = './datasets/alpaca_train_dataset.jsonl'

EOS_TOKEN = tokenizer.eos_token # 必须添加 EOS_TOKEN

_train_features = Features({
    'instrcuct': Value('string'),
    'input': Value('string'),
    'output': Value('string')
})

train_dataset = load_dataset(
    "json", 
    data_files=train_dataset_path, 
    split="train", 
    features=_train_features,
)